In [2]:
from functools import partial
import os
import tempfile
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray import train
from ray.train import Checkpoint, get_checkpoint
from ray.tune.schedulers import ASHAScheduler
import ray.cloudpickle as pickle
from ultralytics import YOLO

In [4]:
TASK2METRIC = {
    "detect": "metrics/mAP50(B)", #detect 지표를 mAP50-95에서 50으로 변경하기 위함
    "segment": "metrics/mAP50-95(M)",
    "classify": "metrics/accuracy_top1",
    "pose": "metrics/mAP50-95(P)",
    "obb": "metrics/mAP50-95(B)",
}

In [5]:
experiment_name = 'tune'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

# Load a YOLOv8n model
best_model_tuning = YOLO("yolov8n.pt")

task = best_model_tuning.task
metric = TASK2METRIC[task] 

result_grid = best_model_tuning.tune(
    data="/home/under1/Detect/jeongui/axial/axial_t1wce_2_class.yaml",
    epochs=200,
    use_ray=True,
    name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'),
    gpu_per_trial=1  # 각 트라이얼에 1개의 GPU 할당
)

(_tune pid=20524) Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
(_tune pid=20524) engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/under1/Detect/jeongui/axial/axial_t1wce_2_class.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=tune_raytune, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save

  0%|          | 0.00/6.25M [00:00<?, ?B/s]
  4%|▍         | 272k/6.25M [00:00<00:02, 2.52MB/s]
 94%|█████████▎| 5.85M/6.25M [00:01<00:00, 4.39MB/s]
100%|██████████| 6.25M/6.25M [00:01<00:00, 4.46MB/s]
 93%|█████████▎| 5.83M/6.25M [00:01<00:00, 4.09MB/s]
100%|██████████| 6.25M/6.25M [00:02<00:00, 3.24MB/s]
 93%|█████████▎| 5.80M/6.25M [00:02<00:00, 4.23MB/s]
100%|██████████| 6.25M/6.25M [00:02<00:00, 2.98MB/s]
100%|██████████| 6.25M/6.25M [00:02<00:00, 2.95MB/s]
train: Scanning /home/under1/Detect/dataset/axial/labels/train.cache... 237 images, 0 backgrounds, 0 corrupt: 100%|██████████| 237/237 [00:00<?, ?it/s]
  0%|          | 0.00/6.25M [00:00<?, ?B/s] [repeated 3x across cluster]
 90%|█████████ | 5.63M/6.25M [00:02<00:00, 4.22MB/s] [repeated 67x across cluster]


(_tune pid=20523) AMP: checks passed ✅
(_tune pid=20525) Overriding model.yaml nc=80 with nc=2 [repeated 3x across cluster]
(_tune pid=20525)                    from  n    params  module                                       arguments                      [repeated 3x across cluster]
(_tune pid=20525)  19                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]               [repeated 21x across cluster]
(_tune pid=20525)   8                  -1  1    460288  ultralytics.nn.modules.block.C2f             [256, 256, 1, True]            [repeated 12x across cluster]
(_tune pid=20525)   9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256, 256, 5]                  [repeated 3x across cluster]
(_tune pid=20525)  13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']           [repeated 6x across cluster]
(_tune pid=20525)  20             [-1, 9]  1         0  ultralytics.n

  0%|          | 0/15 [00:00<?, ?it/s]
val: Scanning /home/under1/Detect/dataset/axial/labels/val.cache... 59 images, 0 backgrounds, 0 corrupt: 100%|██████████| 59/59 [00:00<?, ?it/s] [repeated 7x across cluster]


(_tune pid=20525) 


...

  0%|          | 0/15 [00:00<?, ?it/s]
    200/200       2.1G   0.004108     0.4296     0.9729         16        640:   0%|          | 0/15 [00:00<?, ?it/s]
    200/200       2.1G   0.004219      0.435     0.9602         16        640:  13%|█▎        | 2/15 [00:00<00:01, 12.46it/s]
    200/200       2.1G   0.003969      0.427     0.9697         16        640:  27%|██▋       | 4/15 [00:00<00:00, 11.78it/s]
    199/200       2.1G   0.004148     0.4039     0.9793         16        640:  67%|██████▋   | 10/15 [00:00<00:00, 10.64it/s] [repeated 7x across cluster]
    199/200       2.1G   0.004209     0.4121     0.9791         17        640:  93%|█████████▎| 14/15 [00:01<00:00, 10.62it/s] [repeated 11x across cluster]
    200/200       2.1G   0.004136      0.425     0.9726         17        640:  53%|█████▎    | 8/15 [00:00<00:00, 11.44it/s]
    200/200      2.38G    0.00411     0.4125     0.9663         14        640: 100%|██████████| 15/15 [00:01<00:00, 11.13it/s]
                 Class   

(_tune pid=23978) 
(_tune pid=23978) 200 epochs completed in 0.370 hours.
(_tune pid=23978) Optimizer stripped from runs/detect/tune/tune_raytune/weights/last.pt, 6.3MB
(_tune pid=23978) Optimizer stripped from runs/detect/tune/tune_raytune/weights/best.pt, 6.3MB
(_tune pid=23978) 
(_tune pid=23978) Validating runs/detect/tune/tune_raytune/weights/best.pt...
(_tune pid=23978) Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
(_tune pid=23978) Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/2 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  50%|█████     | 1/2 [00:00<00:00,  7.16it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.42it/s]


(_tune pid=23978)               negative         20         20      0.921       0.95      0.985      0.672
(_tune pid=23978)               positive         39         42       0.87      0.929      0.959      0.643
(_tune pid=23978)                    all         59         62      0.895      0.939      0.972      0.658 [repeated 2x across cluster]
(_tune pid=23978)       Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
(_tune pid=23978) Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 0.9ms postprocess per image
(_tune pid=23978) Results saved to runs/detect/tune/tune_raytune


2024-07-03 15:30:19,973	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/under1/Detect/jeongui/axial/runs/detect/tune/_tune_2024-07-03_15-06-30' in 0.0148s.
2024-07-03 15:30:19,990	INFO tune.py:1041 -- Total run time: 1429.11 seconds (1429.05 seconds for the tuning loop).


In [6]:
for i, result in enumerate(result_grid):
    print(f"Trial #{i}: Configuration: {result.config}, Last Reported Metrics: {result.metrics}")

Trial #0: Configuration: {'lr0': 0.030125282251229703, 'lrf': 0.427491987560607, 'momentum': 0.9413068324938147, 'weight_decay': 0.00018669346966247136, 'warmup_epochs': 3.1807186190495003, 'warmup_momentum': 0.038077007240713306, 'box': 0.16423555031072096, 'cls': 0.8053379613829026, 'hsv_h': 0.07381425763840267, 'hsv_s': 0.434879514250687, 'hsv_v': 0.6756308263872348, 'degrees': 19.618960995939535, 'translate': 0.7334746536003447, 'scale': 0.5598693863272276, 'shear': 8.959438851105592, 'perspective': 0.0006123921190673364, 'flipud': 0.3391408488525104, 'fliplr': 0.7806877168115166, 'bgr': 0.6090540740037854, 'mosaic': 0.49286315916966605, 'mixup': 0.28454503667902187, 'copy_paste': 0.6934120196273316, 'data': '/home/under1/Detect/jeongui/axial/axial_t1wce_2_class.yaml', 'epochs': 200, 'name': 'tune/tune_raytune'}, Last Reported Metrics: {'metrics/precision(B)': 0.43334, 'metrics/recall(B)': 0.63095, 'metrics/mAP50(B)': 0.51104, 'metrics/mAP50-95(B)': 0.22112, 'val/box_loss': 0.04078

In [7]:
# Extract the best trial configuration
best_result = result_grid.get_best_result(metric=metric, mode="max")
best_config = best_result.config
print("Best hyperparameters found were: ", best_config)

Best hyperparameters found were:  {'lr0': 0.09140856692670214, 'lrf': 0.20438845196468455, 'momentum': 0.6644328349501873, 'weight_decay': 0.0009517807614330093, 'warmup_epochs': 4.594276377185559, 'warmup_momentum': 0.4947786111220072, 'box': 0.03707090349942871, 'cls': 0.5394038084690531, 'hsv_h': 0.04210269017809222, 'hsv_s': 0.5785910720471259, 'hsv_v': 0.8029589530559108, 'degrees': 5.845387813046559, 'translate': 0.1858690159885819, 'scale': 0.13351776791072412, 'shear': 6.437119998735074, 'perspective': 0.0008110848517133756, 'flipud': 0.20477242248131233, 'fliplr': 0.3653461954872106, 'bgr': 0.6952333033234865, 'mosaic': 0.6839491914281834, 'mixup': 0.4757145541776924, 'copy_paste': 0.6656779758525682, 'data': '/home/under1/Detect/jeongui/axial/axial_t1wce_2_class.yaml', 'epochs': 200, 'name': 'tune/tune_raytune'}


In [12]:
experiment_name = 'train'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

# 'data' 항목 제거
if 'data' in best_config:
    del best_config['data']
if 'name' in best_config:
    del best_config['name']

# 최고의 하이퍼파라미터로 모델 재훈련
best_model = YOLO("yolov8n.pt")  # 모델 재초기화
best_results = best_model.train(data="axial_t1wce_2_class.yaml",
                                optimizer='SGD',
                                augment=True,
                                name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'),
                                **best_config)

Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=axial_t1wce_2_class.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train_raytune2, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_c

train: Scanning /home/under1/Detect/dataset/axial/labels/train.cache... 237 images, 0 backgrounds, 0 corrupt: 100%|██████████| 237/237 [00:00<?, ?it/s]
val: Scanning /home/under1/Detect/dataset/axial/labels/val.cache... 59 images, 0 backgrounds, 0 corrupt: 100%|██████████| 59/59 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/train_raytune2/labels.jpg... 
optimizer: SGD(lr=0.09140856692670214, momentum=0.6644328349501873) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0009517807614330093), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train/train_raytune2
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/200      2.56G   0.008602      4.297      1.574         29        640: 100%|██████████| 15/15 [00:01<00:00,  8.81it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.41it/s]

                   all         59         62     0.0021      0.608     0.0245    0.00323


...

    200/200      2.38G   0.004444     0.4943     0.9992         14        640: 100%|██████████| 15/15 [00:01<00:00, 11.69it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  8.54it/s]

                   all         59         62      0.915      0.855      0.948      0.603



200 epochs completed in 0.131 hours.
Optimizer stripped from runs/detect/train/train_raytune2/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/train_raytune2/weights/best.pt, 6.3MB

Validating runs/detect/train/train_raytune2/weights/best.pt...
Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.50it/s]


                   all         59         62      0.828      0.879      0.921      0.607
              negative         20         20       0.74        0.9      0.882      0.618
              positive         39         42      0.916      0.857       0.96      0.596
Speed: 0.1ms preprocess, 2.4ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/train/train_raytune2


In [13]:
# train
best_results.results_dict

{'metrics/precision(B)': 0.8281072662322662,
 'metrics/recall(B)': 0.8785714285714286,
 'metrics/mAP50(B)': 0.9210038513831792,
 'metrics/mAP50-95(B)': 0.6071716297274001,
 'fitness': 0.638554851892978}

In [14]:
# val
experiment_name = 'val'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

best_model = YOLO("/home/under1/Detect/jeongui/axial/runs/detect/train/train_raytune2/weights/best.pt")
best_model.val(name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'))

Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/under1/Detect/dataset/axial/labels/val.cache... 59 images, 0 backgrounds, 0 corrupt: 100%|██████████| 59/59 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  6.23it/s]


                   all         59         62      0.853      0.862      0.924      0.626
              negative         20         20      0.734        0.9      0.881      0.623
              positive         39         42      0.972      0.823      0.967      0.629
Speed: 0.1ms preprocess, 1.3ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/val/val_raytune2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f0db4525870>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [15]:
# test
experiment_name = 'test'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

best_model.val(name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'), split="test")

Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)


val: Scanning /home/under1/Detect/dataset/axial/labels/test.cache... 75 images, 0 backgrounds, 0 corrupt: 100%|██████████| 75/75 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  4.96it/s]


                   all         75         81      0.754      0.802       0.83      0.539
              negative         75         81      0.754      0.802       0.83      0.539
Speed: 0.2ms preprocess, 1.7ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/test/test_raytune2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f0db44b8c70>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 